**Resumo do que abordaremos hoje:**

- Árvores
    - Quais são os parâmetros mais sensíveis
    - Função de split
    - Critérios de parada
    - Notas sobre minha experiência pessoal
- Ensembles
    - Bagging
- Viés e variância
     - Demonstrar quando o bagging funciona e falha, de acordo com a variância do preditor
- Random Patches
- Random Forest

# 1. Árvores de decisão

- Dividir para conquistar
- Estruturas hierárquicas
- Construção recursiva
- Seleção de features:
    - Quanto mais acima na árvore, mais importante
    - O caminho da raíz à folha também pode ser utilizado como indicativo de importância
- Interpretabilidade



In [ ]:
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

## 1.1. Split na prática

Vamos relembrar como os splits em uma árvore de decisão ocorrem na prática.

Primeiro, vou escolher uma heurística de classificação para fazermos o teste.

Por simplicidade, utilizarei a entropia e o ganho de informação:

$E = -\sum_i^C p_i\log p_i$

(caso com splits binários)

$IG = E(y) - \dfrac{|y_L|}{|y|}E(y_L) - \dfrac{|y_R|}{|y|}E(y_R)$

In [ ]:
import numpy as np


def entropy(y):
    n = len(y)
    _, cnts = np.unique(y, return_counts=True)

    entropy = 0
    for c in cnts:
        p_i = c / n
        entropy += (p_i * np.log2(p_i))
    return -entropy

def info_gain(x, y, limiar):
    n = len(y)
    y_l = y[x <= limiar]
    y_r = y[x > limiar]

    return entropy(y) - (len(y_l) / n) * entropy(y_l) - (len(y_r) / n) * entropy(y_r)

Hora de testar!

In [ ]:
rng = np.random.default_rng(7)

x = rng.uniform(-3, 3, 100)
y = np.array(
    [0 if xi <= 0 else 1 for xi in x]
)
x[:3], y[:3]

Para fazer um split numérico, a opção mais usual é testar todos os valores disponíveis:

In [ ]:
def testa_todos(x, y, heuristica):
    ordenados = sorted(x)

    candidatos = [
        (limiar, heuristica(x, y, limiar)) for limiar in ordenados
    ]

    return candidatos

Vamos olhar nossos candidatos a split:

In [ ]:
candidatos_split = testa_todos(x, y, info_gain)
candidatos_split

In [ ]:
def melhor_split(candidatos):
    return max(candidatos, key=lambda c: c[1])

In [ ]:
melhor_split(candidatos_split)

Nossa implementação sempre selecionará o primeiro "melhor candidato", no caso de empates.

Que tal olharmos um caso menos trivial?

In [ ]:
y = np.array(
    [0 if xi <= -2 or (xi > 0 and xi <= 0.5) or (xi > 1.5) else 1 for xi in x]
)
candidatos_split = testa_todos(x, y, info_gain)
melhor_split(candidatos_split)

Agora já sabemos como encontrar o melhor split, dada uma feature.

Vamos fingir que nosso problema tivesse apenas uma feature. Vamos construir árvore com base em nossas ferramentas.

In [ ]:
import matplotlib.pyplot as plt


def plot_1dtree(x, y, heuristica, min_sample_split):
    queue = [(x, y, 1)]

    markers = {
        0: "o",
        1: "^"
    }

    colors = {
        0: "blue",
        1: "black"
    }
    for c in np.unique(y):
        x_aux = x[y == c]
        plt.scatter(x_aux, [0 for _ in range(len(x_aux))], c=colors[c], marker=markers[c])


    while len(queue) > 0:
        x_aux, y_aux, alt = queue.pop(0)

        # Número minimo de amostras
        if len(x_aux) < min_sample_split:
            continue
        
        # Partição homogênea
        if len(np.unique(y_aux)) == 1:
            continue

        limiar, ig = melhor_split(testa_todos(x_aux, y_aux, heuristica))

        plt.axvline(limiar, ymax=alt)

        x_l, y_l = x_aux[x_aux <= limiar], y_aux[x_aux <= limiar]
        queue.append((x_l, y_l, alt * 0.9))
        x_r, y_r = x_aux[x_aux > limiar], y_aux[x_aux > limiar]
        queue.append((x_r, y_r, alt * 0.9))
    
    plt.show()

In [ ]:
plot_1dtree(x, y, info_gain, 2)

In [ ]:
y = rng.choice([0, 1], p=[0.5, 0.5], size=len(x))

In [ ]:
plot_1dtree(x, y, info_gain, 2)

In [ ]:
plot_1dtree(x, y, info_gain, 10)

In [ ]:
plot_1dtree(x, y, info_gain, 20)

### 1.1.1. E regressão, como fica?

Em regressão temos um target continuo. Logo, outros tipos de heurísticas devem ser utilizadas.

Ilustrarei uma das heurísticas, que é equivalente à operação padrão utilizada no `sklearn`.

Redução de variância:

$VR = Var(y) - \dfrac{|y_l|}{|y|}Var(y_l) - \dfrac{|y_r|}{|y|}Var(y_r)$

In [ ]:
def vr(x, y, limiar, min_sample_split=5):
    n = len(y)
    vr = np.var(y, ddof=1)
    y_l = y[x <= limiar]
    y_r = y[x > limiar]

    if len(y_l) < min_sample_split or len(y_r) < min_sample_split:
        return 0

    vr -= (len(y_l) / n) * np.var(y_l, ddof=1)
    vr -= (len(y_r) / n) * np.var(y_r, ddof=1)

    return vr


In [ ]:
y = np.zeros(len(x))

y[x <= 0] = rng.normal(-1, 0.1, size=len(np.where(x <= 0)[0]))
y[x > 0] = rng.normal(1, 0.1, size=len(np.where(x > 0)[0]))

In [ ]:
split = melhor_split(testa_todos(x, y, vr))

In [ ]:
plt.scatter(x, y)
plt.axvline(split[0], ymin=min(y), ymax=max(y), c="red")

## 1.2. Notas sobre as árvores no sklearn

Alguns insights que obtive com a experiência.

### 1.2.1. As árvores não são determinísticas

O sklearn implementa (de forma muito rápida e elegante) as suas estruturas de árvore. De fato, todos os algoritmos de ensemble compartilham do mesmo preditor base.

Para usar a mesma árvore por si só, nas Random Forest e Extra Trees, além de todos os algorítmos de boosting, devem existir prós e contras.

- Por padrão, o sklearn utiliza todas as features para avaliar splits
    - No entanto, devido aos ensembles, subsets das features podem ser utilizadas
- As features são sempre embaralhadas por padrão
    - Se duas features são igualmente "boas" (de acordo com o critério de split) elas podem ser permutadas entre múltiplas execuções
- **Solução:** Sempre fixar a seed de geração de números aleatórios (`random_state`)

In [ ]:
from sklearn.tree import export_text

rng = np.random.default_rng(7)
X = np.zeros((1000, 4))

X[:, 0] = rng.normal(0, 1, 1000)
X[:, 1] = rng.normal(1, 3, 1000)
X[:, 2] = X[:, 0]
X[:, 3] = X[:, 1]

y = rng.choice([0, 1], size=1000, p=[0.3, 0.7])


In [ ]:
dt = DecisionTreeClassifier(random_state=7, max_depth=2)
dt.fit(X, y)

r = export_text(dt, feature_names=["A", "B", "C", "D"])
print(r)

In [ ]:
dt = DecisionTreeClassifier(random_state=10001, max_depth=2)
dt.fit(X, y)

r = export_text(dt, feature_names=["A", "B", "C", "D"])
print(r)

### 1.2.2. A configuração padrão do sklearn tem grandes chances de gerar overfitting

Por padrão, o `sklearn` não restringe em basicamente nada as suas árvores.
Não tem problema algum nisso, mas devemos ficar "espertos" para utilizar as árvores.

**Exemplo:**

In [ ]:
from sklearn.datasets import load_digits
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

data = load_digits()

X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=42, shuffle=True)

In [ ]:
dt = DecisionTreeClassifier(random_state=42)

dt.fit(X_train, y_train)

accuracy_score(y_train, dt.predict(X_train)), accuracy_score(y_test, dt.predict(X_test))

In [ ]:
dt.get_depth()

In [ ]:
dt = DecisionTreeClassifier(random_state=42, max_depth=10)

dt.fit(X_train, y_train)

accuracy_score(y_train, dt.predict(X_train)), accuracy_score(y_test, dt.predict(X_test))

In [ ]:
dt.get_depth()

## 1.3. Ajuste de hiper-parâmetros

As árvores contam com vários parâmetros para ajuste. Mas quais são mais interessantes de focar?

>  Em geral, queremos restringir as árvores (especialmente de uma forma "esperta") para que possamos evitar overfitting.

Dentre os parâmetros que valem a pena checar:

- `max_depth`
- `min_samples_split`
- `min_samples_leaf`
- `min_weight_fraction_leaf`
- `max_leaf_nodes`
- `min_impurity_decrease`
- `ccp_alpha`

Várias possibilidades!

Na minha humilde opinião, nem todos precisam ser ajustados ao mesmo tempo. Em minha visão, esses hiper-parâmetros poderiam ser agrupados. Uma possibilidade seria ajustar um grupo de parâmetros, ou até mesmo uma combinação de grupos.

Eis aqui a minha visão pessoal:

1. Pré-poda:
    - **A:** limitação "cega" - não leva em conta as heurísticas de split
        - `max_depth`, `max_leaf_nodes`
    - **B:** limitação que não leva as heurísticas de split diretamente em conta
        - `min_samples_leaf`, `min_samples_split`, `min_weight_fraction_leaf`
    - **C:** limitação que considera as heurísticas de split
        - `min_impurity_decrease`
2. Pós-poda:
    - `ccp_alpha`

---

Questão para reflexão:

> Qual a diferença prática entre as estruturas geradas por uma árvore restringida via altura máxima e as outras estratégias?

---

De resto, as técnicas de ajuste de hiper-parâmetro se aplicam como usual.

**Obs:**

> É possível dar pesos diferentes para as classes -> cenários com desbalanceamento, por exemplo.

# 2. Viés e variância revisitados: ensembles baseados em Bagging

- Redução da variância
- Vimos no começo do curso que a variância e o viés se contrapoem:
    - Se aumentamos o viés, diminuímos a variância. E vice-versa.

- Viés alto -> underfitting
- Variância alta -> overfitting

A princípio, overffiting parece algo sempre "maléfico", mas no quesito dos ensembles baseados em bagging, esse sobreajuste pode vir a calhar.

Por que as árvores de decisão são tão populares em ensembles?

In [ ]:
data = load_digits()

X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=42, shuffle=True)

dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)

accuracy_score(y_test, dt.predict(X_test)), dt.get_depth(), dt.get_n_leaves()

In [ ]:
data = load_digits()

X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=43, shuffle=True)

dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)

accuracy_score(y_test, dt.predict(X_test)), dt.get_depth(), dt.get_n_leaves()

In [ ]:
data = load_digits()

X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=44, shuffle=True)

dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)

accuracy_score(y_test, dt.predict(X_test)), dt.get_depth(), dt.get_n_leaves()

Por que isso acontece? Vamos voltar ao nosso exemplo lá do início do plantão:

In [ ]:
rng = np.random.default_rng(7)

x = rng.uniform(-3, 3, 100)
y = np.array(
    [0 if xi <= 0 else 1 for xi in x]
)

In [ ]:
plot_1dtree(x, y, info_gain, 2)

Deixa eu bagunçar um pouco esses dados:

In [ ]:
np.where(y==0)

In [ ]:
# Selecionei 3 posições arbitrárias para permutar as labels
y[np.array([9, 87, 94])] = 1

In [ ]:
plot_1dtree(x, y, info_gain, 2)

Desafio:

> O que está acontecendo?

E se restringirmos as coisas (nesse caso, restringirmos bastante)?

In [ ]:
plot_1dtree(x, y, info_gain, 10)

In [ ]:
plot_1dtree(x, y, info_gain, 20)

In [ ]:
plot_1dtree(x, y, info_gain, 60)

Uma pequena mudança nos dados acarreta grandes mudanças no modelo gerado!

Alta variância! :)

---

Que lições podemos tomar disso já de antemão se formos pensar em ensembles com bagging?

> Pode ser uma boa ideia deixar as árvores em um ensemble baseado em bagging "voarem livres" ou com pouca restrição.

**Por quê?**

---

Note que a Random Forest e outros ensembles similares utilizam mais de uma estratégia para induzir diversidade entre seus membros.

# 3. Comparando o efeito de bagging em algoritmos de AM

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import KFold, cross_val_score

In [ ]:
dataset = load_digits()

kf = KFold(10, random_state=42, shuffle=True)

## 3.1. Árvores de decisão

Vamos avaliar o efeito do bagging em algumas famílias de algoritmos preditivos.

### 3.1.1. Individualmente

In [ ]:
accs = cross_val_score(
    DecisionTreeClassifier(random_state=7),
    X=dataset.data,
    y=dataset.target,
    cv=kf,
    scoring="accuracy"
)
np.mean(accs), np.std(accs)

In [ ]:
accs = cross_val_score(
    DecisionTreeClassifier(random_state=7, min_impurity_decrease=0.001),
    X=dataset.data,
    y=dataset.target,
    cv=kf,
    scoring="accuracy"
)
np.mean(accs), np.std(accs)

### 3.1.2. Bagging

Sem restrições

In [ ]:
accs = cross_val_score(
    BaggingClassifier(
        DecisionTreeClassifier(random_state=7),
        random_state=42
    ),
    X=dataset.data,
    y=dataset.target,
    cv=kf,
    scoring="accuracy"
)
np.mean(accs), np.std(accs)

Restringindo a profundidade máxima da árvore

In [ ]:
accs = cross_val_score(
    BaggingClassifier(
        DecisionTreeClassifier(random_state=7, max_depth=5),
        random_state=42
    ),
    X=dataset.data,
    y=dataset.target,
    cv=kf,
    scoring="accuracy"
)
np.mean(accs), np.std(accs)

**Desafio:**

> O que está acontecendo?

## 3.2 Redes Neurais

Não irei me aprofundar nas configurações das redes de forma alguma. O objetivo aqui é apenas ilustrativo.

In [ ]:
from sklearn.neural_network import MLPClassifier

### 3.2.1. Individualmente

In [ ]:
accs = cross_val_score(
    MLPClassifier(random_state=1),
    X=dataset.data,
    y=dataset.target,
    cv=kf,
    scoring="accuracy"
)
np.mean(accs), np.std(accs)

In [ ]:
accs = cross_val_score(
    MLPClassifier(random_state=2),
    X=dataset.data,
    y=dataset.target,
    cv=kf,
    scoring="accuracy"
)
np.mean(accs), np.std(accs)

### 3.2.2. Bagging

In [ ]:
accs = cross_val_score(
    BaggingClassifier(
        MLPClassifier(random_state=1),
        random_state=42
    ),
    X=dataset.data,
    y=dataset.target,
    cv=kf,
    scoring="accuracy"
)
np.mean(accs), np.std(accs)

## 3.3. k-NN

Quais são as apostas para o k-NN?

### 3.3.1. Individualmente

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

accs = cross_val_score(
    KNeighborsClassifier(),
    X=dataset.data,
    y=dataset.target,
    cv=kf,
    scoring="accuracy"
)
np.mean(accs), np.std(accs)

### 3.3.2. Bagging

In [ ]:
accs = cross_val_score(
    BaggingClassifier(
        KNeighborsClassifier(),
        random_state=42
    ),
    X=dataset.data,
    y=dataset.target,
    cv=kf,
    scoring="accuracy"
)
np.mean(accs), np.std(accs)

## 3.4. Naive Bayes

### 3.4.1. Individualmente

In [ ]:
from sklearn.naive_bayes import GaussianNB


accs = cross_val_score(
    GaussianNB(),
    X=dataset.data,
    y=dataset.target,
    cv=kf,
    scoring="accuracy"
)
np.mean(accs), np.std(accs)

### 3.4.2. Bagging

In [ ]:
accs = cross_val_score(
    BaggingClassifier(
        GaussianNB(),
        random_state=42
    ),
    X=dataset.data,
    y=dataset.target,
    cv=kf,
    scoring="accuracy"
)
np.mean(accs), np.std(accs)

## 3.5. Regressão Logistica

Façam suas apostas!

### 3.5.1. Individualmente


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


accs = cross_val_score(
    Pipeline([
        ('scaler', StandardScaler()),
        ('model', LogisticRegression(random_state=1, max_iter=200))
    ]),
    X=dataset.data,
    y=dataset.target,
    cv=kf,
    scoring="accuracy"
)
np.mean(accs), np.std(accs)

### 3.5.2. Bagging

In [ ]:
accs = cross_val_score(
    Pipeline([
        ('scaler', StandardScaler()),
        ('model', BaggingClassifier(LogisticRegression(random_state=1, max_iter=200), random_state=42))
    ]),
    X=dataset.data,
    y=dataset.target,
    cv=kf,
    scoring="accuracy"
)
np.mean(accs), np.std(accs)

# 4. Random Patches

Até o momento, nós focamos em apenas uma estratégia de perturbação nos dados:

> Bagging

Induzir diversidade.

Existem mais formas de fazer isso. Outra forma muito popular é a utilização de sub-conjuntos de features para induzir cada preditor.

**Atenção:**

- Bagging: amostragem com reposição
- Sub-conjuntos de features: amostragem sem reposição (abordagem mais usual)

---
Qual é o efeito de sub-amostrarmos features?

- O que acontece em uma árvore de decisão?
- O que acontece no k-NN?
- Quando isso pode ser mais ou menos efetivo?

---

A combinação de bagging e sub-conjuntos de features por preditor base resulta no algorítmo chamado: *Random Patches*.

**Atenção (de novo):**

> A sub-amostragem de features por preditor base é **global** no Random Patches.

Vamos ver na prática:

## 4.1. Árvores de Decisão + Bagging


In [ ]:
accs = cross_val_score(
    BaggingClassifier(
        DecisionTreeClassifier(random_state=1),
        random_state=42
    ),
    X=dataset.data,
    y=dataset.target,
    cv=kf,
    scoring="accuracy"
)
np.mean(accs), np.std(accs)

## 4.2. Árvores de Decisão + Random Patches

In [ ]:
accs = cross_val_score(
    BaggingClassifier(
        DecisionTreeClassifier(random_state=1),
        random_state=42,
        max_features=round(0.6 * dataset.data.shape[1])
    ),
    X=dataset.data,
    y=dataset.target,
    cv=kf,
    scoring="accuracy"
)
np.mean(accs), np.std(accs)

# 5. Random Forest

O Random Patches, por fazer sub-amostragem de features a nível global, pode ser aplicado a qualquer algoritmo.

A Random Forest vai um passo além, no quesito indução de diversidade.

Ao invés de subamostrar features globalmente, as RFs

> Realização a sub-amostragem de features a nível **local**

Mas como?

Durante cada tentativa de split, em cada nó, um novo sub-conjunto de features é amostrado (sem reposição).

---

**Desafio:**

> Qual a vantagem de se fazer isso?

---

Em contrapartida, agora estamos limitados a árvores de decisão.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

accs = cross_val_score(
    RandomForestClassifier(
        random_state=42,
        max_features=round(0.6 * dataset.data.shape[1]),
        n_estimators=10
    ),
    X=dataset.data,
    y=dataset.target,
    cv=kf,
    scoring="accuracy"
)
np.mean(accs), np.std(accs)

In [ ]:
accs = cross_val_score(
    RandomForestClassifier(
        random_state=42,
        n_estimators=100
    ),
    X=dataset.data,
    y=dataset.target,
    cv=kf,
    scoring="accuracy"
)
np.mean(accs), np.std(accs)

## 5.1. E agora? E para tunar?

As RFs contam com vários parâmetros por si só, e ainda trazem os parâmetros dos modelos base (árvores). E agora?

Aqui vai uma pitada da minha experiência pessoal. O que focar?

- `n_estimators`: no entanto, existe um ponto de saturação
    - mais árvores != mais acurácia. Isso é até bom!
- `max_features`: eu diria que esse é o parâmetro mais sensível

O paper do Breiman introduzindo as RFs foca nesses dois parâmetros.

Desconheço a existência de consenso quanto as vantagens (ou desvantagens) de se podar as árvores na floresta. O que eu faço normalmente? Deixo as árvores crescerem sem restrições.

Um coisa incrível sobre as RFs? O desempenho delas normalmente não varia muito após o ajuste de hiper-parâmetros.

**Dica:**

Ao se deparar com um novo problema eu costumo:

1. Testar um baseline simples
2. Testar um modelo linear
3. Testar uma RF com os hiper-parâmetros padrão

## 5.2. Feature importance

Existem algumas maneiras de se estimar a importância das features a partir de uma floresta de árvores. Por padrão, o `sklearn` utiliza um algoritmo de calculo de importância baseado na some do decréscimo médio na impureza de cada árvore.

Para utilizá-lo:



In [ ]:
from sklearn.datasets import load_iris

dataset = load_iris()

# Não me julguem por usar o conjunto inteiro, o propósito é fazer algo didático
rf = RandomForestClassifier(random_state=42)
rf.fit(dataset.data, dataset.target)

In [ ]:
import pandas as pd

importances = rf.feature_importances_

# Opcional, mas interessante
std = np.std([tree.feature_importances_ for tree in rf.estimators_], axis=0)

importances = pd.Series(importances, index=dataset.feature_names)

fig, ax = plt.subplots()
importances.plot.bar(yerr=std, ax=ax)
ax.set_title("Feature importance")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()

Eu vou descrever uma alternativa mais custosa, porém mais confiável. Esse algoritmo para cálculo de importância das features é descrito no artigo do Breiman, onde as Random Forest (também) foram introduzidas.

---

O Bagging nos dá algo de brinde:

- Cada árvore tem um conjunto de amostras que nunca foi utilizado para treino: amostras Out-of-Bag (OOB).
- Com as amostras OOB podemos calcular OOBE (e essa estimativa não é enviesada, como treinar e testar com os mesmos dados!)

Com esses ingredientes, podemos dar um passo além e estimarmos uma medida de importância de feature:

**RF Feature Importance (Permutation):**

1. Para cada feature:
    1. Calcule o OOBE
    2. Permute os valores da feature aleatoriamente
    3. Recalcule o OOBE para os dados permutados
    4. Calcule a diferença
2. (Opcional) Normalize os valores obtidos

O `sklearn` não implementa essa opção diretamente. A implementação padrão de RF no R tem essa opção.

No entanto, podemos simular algo parecido essa estratégia utilizando a função `permutation_importance` ([link](https://scikit-learn.org/stable/modules/generated/sklearn.inspection.permutation_importance.html)), que está disponível no módulo `inspection` do `sklearn`. No entanto, precisamos passar um conjunto de testes para o cálculo das importâncias.

## 5.3. E dá para "bagunçar" mais?

Até agora a nossa "receita" de ensemble inclui:

- `n_estimators` preditores com alta variância (de preferência árvores de decisão)
- bagging
- sub-amostragem (local, no caso de árvores) de features

No caso da RF, temos que em cada split um novo su-conjunto de features é amostrado e o melhor ponto, dentre a melhor das features é selecionada para criar um nó de decisão.

Como poderíamos deixar mais aleatório esse processo?

> Extra Trees!

- Usa sub-conjuntos de features assim como a RF, no entanto, os pontos de split também são aleatórios!
- Escolhe o melhor ponto aleatório de split no sub-conjunto de features selecionadas 

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

accs = cross_val_score(
    ExtraTreesClassifier(
        random_state=42
    ),
    X=dataset.data,
    y=dataset.target,
    cv=kf,
    scoring="accuracy"
)
np.mean(accs), np.std(accs)

Por padrão, as Extra Trees não usam Bagging (bootstrap sampling). Mas esse padrão pode ser modificado:

In [ ]:
accs = cross_val_score(
    ExtraTreesClassifier(
        random_state=42,
        bootstrap=True
    ),
    X=dataset.data,
    y=dataset.target,
    cv=kf,
    scoring="accuracy"
)
np.mean(accs), np.std(accs)

Qual é o impacto da mudança na estratégia de particionamento das árvores?

In [ ]:
from sklearn.datasets import make_classification

X, y = make_classification(
    n_samples=5000, n_features=20, n_classes=5, n_informative=15,
    random_state=8
)

In [ ]:
%%timeit

rf = RandomForestClassifier(random_state=42)
rf.fit(X, y)

In [ ]:
%%timeit

xt = ExtraTreesClassifier(random_state=42)
xt.fit(X, y)